In [2]:
pip install pyTelegramBotAPI mysql-connector-python openai mysql-connector


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 111.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 16.6 MB/s eta 0:00:00
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp312-cp312-linux_x86_64.whl size=247950 sha256=b0fd7ab83825a5856eee380909bdeeeffe8ded1db3a1217ec996ff197842428f
  Stored in directory: /root/.cache/pip/wheels/03/17/fa/d7604c72dd3dd6d3eb3d249abf36cc532c9a9b4354b8f1bc4f
Successfully built mysql-connector


In [ ]:
import telebot
from telebot.types import ReplyKeyboardMarkup, InlineKeyboardMarkup, InlineKeyboardButton
import sqlite3
import time
from openai import OpenAI

# ============================
# CONFIG
# ============================

BOT_TOKEN = "bot_token"
OPENAI_KEY = "api_key"

bot = telebot.TeleBot(BOT_TOKEN, parse_mode="HTML")
client = OpenAI(api_key=OPENAI_KEY)

db = sqlite3.connect("universities.db", check_same_thread=False)
cursor = db.cursor()

FLAGS = {"USA": "🇺🇸", "South Korea": "🇰🇷"}

# ChatGPT mode on/off per user
user_chat_mode = {}
# Temporary data (country, gpa, ielts) per user
user_temp_data = {}


# ============================
# CHATGPT HANDLER
# ============================
# This handler should ONLY run when chat mode is ON,
# and must NOT steal menu messages.
@bot.message_handler(
    func=lambda msg: bool(user_chat_mode.get(msg.chat.id))
    and isinstance(msg.text, str)
    and msg.text not in ["Find University 🎓", "Help ❓"]
)
def chatgpt(message):
    uid = message.chat.id

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are ChatGPT inside a university bot."},
                {"role": "user", "content": message.text}
            ]
        )

        # Correct access for new OpenAI Python library
        reply_text = completion.choices[0].message.content
        bot.send_message(uid, reply_text)

    except Exception as e:
        bot.send_message(uid, f"❌ Error: {str(e)}")


# ============================
# /START MENU
# ============================
@bot.message_handler(commands=['start'])
def start(message):
    chat_id = message.chat.id
    user_chat_mode[chat_id] = False
    user_temp_data.pop(chat_id, None)

    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add("Find University 🎓", "Help ❓")
    bot.send_message(chat_id, "Welcome! Choose an option:", reply_markup=markup)


@bot.message_handler(func=lambda msg: msg.text in ["Find University 🎓", "Help ❓"])
def menu(message):
    chat_id = message.chat.id

    if message.text == "Find University 🎓":
        # When starting a new search, ChatGPT mode must be OFF
        user_chat_mode[chat_id] = False
        user_temp_data.pop(chat_id, None)

        markup = ReplyKeyboardMarkup(resize_keyboard=True)
        markup.add("USA 🇺🇸", "South Korea 🇰🇷")
        msg = bot.send_message(chat_id, "Select your country:", reply_markup=markup)
        bot.register_next_step_handler(msg, ask_country)

    else:
        # Help text, do not change chat mode here
        bot.send_message(
            chat_id,
            "This bot helps you find universities based on your GPA, IELTS, budget,\n"
            "and then you can chat with AI about universities or anything else. 😊"
        )


# ============================
# INPUT: COUNTRY → GPA → IELTS → BUDGET
# ============================
def ask_country(message):
    chat_id = message.chat.id
    user_chat_mode[chat_id] = False  # ensure not in chat mode here

    # Remove flags if user pressed button
    country = message.text.replace("🇺🇸", "").replace("🇰🇷", "").strip()

    if country not in FLAGS:
        msg = bot.send_message(chat_id, "Please select a valid country (use the keyboard buttons).")
        bot.register_next_step_handler(msg, ask_country)
        return

    msg = bot.send_message(chat_id, "Enter your GPA:")
    bot.register_next_step_handler(msg, ask_gpa, country)


def ask_gpa(message, country):
    chat_id = message.chat.id
    user_chat_mode[chat_id] = False  # still in structured flow

    try:
        raw = message.text.replace(",", ".")
        gpa = float(raw)

        # If user gave percentage
        if 50 <= gpa <= 100:
            converted = round((gpa / 100) * 4, 2)
            bot.send_message(
                chat_id,
                f"📘 Detected percentage GPA ({gpa}%). Converted to: <b>{converted}</b>"
            )
            gpa = converted

        # Invalid range
        elif gpa < 1.0 or gpa > 4.0:
            msg = bot.send_message(
                chat_id,
                "😄 Please enter a valid GPA:\n"
                "• 1.0–4.0 (GPA)\n"
                "• or 50–100 (percentage)"
            )
            bot.register_next_step_handler(msg, ask_gpa, country)
            return

        msg = bot.send_message(chat_id, "Enter your IELTS:")
        bot.register_next_step_handler(msg, ask_ielts, country, gpa)

    except Exception:
        msg = bot.send_message(chat_id, "❌ Invalid GPA format! Try like: 3.4 or 85")
        bot.register_next_step_handler(msg, ask_gpa, country)


def ask_ielts(message, country, gpa):
    chat_id = message.chat.id
    user_chat_mode[chat_id] = False

    try:
        ielts = float(message.text.replace(",", "."))

        if ielts > 9 or ielts <= 0:
            msg = bot.send_message(
                chat_id,
                "❌ IELTS must be between 1.0 and 9.0.\n"
                "Example: 5.5, 6.0, 7.5"
            )
            bot.register_next_step_handler(msg, ask_ielts, country, gpa)
            return

        user_temp_data[chat_id] = (country, gpa, ielts)
        msg = bot.send_message(chat_id, "Enter your budget (USD):")
        bot.register_next_step_handler(msg, fetch_universities, country, gpa, ielts)

    except Exception:
        msg = bot.send_message(
            chat_id,
            "❌ Invalid IELTS format! Enter like: 5.5, 6.0, 7.5"
        )
        bot.register_next_step_handler(msg, ask_ielts, country, gpa)


# ============================
# REPEAT BUDGET OPTION
# ============================
def repeat_budget(message):
    chat_id = message.chat.id
    user_chat_mode[chat_id] = False

    try:
        budget = float(message.text.replace(",", "."))
    except Exception:
        msg = bot.send_message(chat_id, "Invalid amount. Enter budget again:")
        bot.register_next_step_handler(msg, repeat_budget)
        return

    # Make sure we still have stored data
    if chat_id not in user_temp_data:
        bot.send_message(chat_id, "Session expired. Please start again with /start.")
        return

    country, gpa, ielts = user_temp_data[chat_id]
    fetch_universities_manual(message, country, gpa, ielts, budget)


# ============================
# FETCH UNIVERSITIES
# ============================
def fetch_universities(message, country, gpa, ielts):
    chat_id = message.chat.id
    user_chat_mode[chat_id] = False

    try:
        budget = float(message.text.replace(",", "."))
    except Exception:
        msg = bot.send_message(chat_id, "Invalid budget! Enter again:")
        bot.register_next_step_handler(msg, fetch_universities, country, gpa, ielts)
        return

    fetch_universities_manual(message, country, gpa, ielts, budget)


def fetch_universities_manual(message, country, gpa, ielts, budget):
    chat_id = message.chat.id
    user_chat_mode[chat_id] = False

    bot.send_message(chat_id, "🔍 Searching universities...")

    query = """
        SELECT unv_name, tuition_fee, ranking, application_page
        FROM universities
        WHERE country=? AND min_gpa<=? AND min_ielts<=? AND tuition_fee<=?
        ORDER BY ranking ASC
        LIMIT 10;
    """

    cursor.execute(query, (country, gpa, ielts, budget))
    rows = cursor.fetchall()

    # No results
    if not rows:
        send_no_results(message, country, gpa, ielts, budget)
        return

    flag = FLAGS.get(country, "")

    for unv_name, fee, ranking, link in rows:
        text = (
            f"{flag} <b>{unv_name}</b>\n"
            f"💰 Tuition: ${fee}\n"
            f"🏆 Ranking: {ranking}"
        )
        markup = InlineKeyboardMarkup()
        markup.add(InlineKeyboardButton("📨 Apply Now", url=link))
        bot.send_message(chat_id, text, reply_markup=markup)

    # After showing results, enable ChatGPT mode
    bot.send_message(chat_id, "💬 <b>You can now chat with AI.</b>")
    user_chat_mode[chat_id] = True


# ============================
# NO RESULTS DIAGNOSTICS
# ============================
def send_no_results(message, country, gpa, ielts, budget):
    chat_id = message.chat.id
    user_chat_mode[chat_id] = False

    problems = []

    cursor.execute(
        "SELECT MIN(min_gpa) FROM universities WHERE country=?",
        (country,)
    )
    min_gpa_required = cursor.fetchone()[0]
    if min_gpa_required is not None and gpa < min_gpa_required:
        problems.append(f"📘 GPA too low! Min: <b>{min_gpa_required}</b>")

    cursor.execute(
        "SELECT MIN(min_ielts) FROM universities WHERE country=?",
        (country,)
    )
    min_ielts_required = cursor.fetchone()[0]
    if min_ielts_required is not None and ielts < min_ielts_required:
        problems.append(f"📗 IELTS too low! Min: <b>{min_ielts_required}</b>")

    cursor.execute(
        "SELECT MIN(tuition_fee) FROM universities WHERE country=?",
        (country,)
    )
    cheapest = cursor.fetchone()[0]
    if cheapest is not None and budget < cheapest:
        problems.append(f"💰 Budget too low! Cheapest: <b>${cheapest}</b>")

    if not problems:
        problems.append("No universities match your criteria. Try changing your inputs.")

    msg_text = "⚠️ <b>No matching universities found.</b>\n\n" + ("\n\n".join(problems))

    markup = InlineKeyboardMarkup()
    markup.add(
        InlineKeyboardButton("🔼 Change Budget", callback_data="increase_budget"),
        InlineKeyboardButton("📝 Restart", callback_data="restart"),
        InlineKeyboardButton("💬 Ask ChatGPT", callback_data="ask_ai")
    )

    bot.send_message(chat_id, msg_text, reply_markup=markup)


# ============================
# CALLBACK HANDLER
# ============================
@bot.callback_query_handler(func=lambda call: True)
def callback_buttons(call):
    chat_id = call.message.chat.id

    if call.data == "increase_budget":
        user_chat_mode[chat_id] = False
        msg = bot.send_message(chat_id, "🔼 Enter your new budget:")
        bot.register_next_step_handler(msg, repeat_budget)

    elif call.data == "restart":
        user_chat_mode[chat_id] = False
        user_temp_data.pop(chat_id, None)
        bot.send_message(chat_id, "🔄 Restarting...")
        start(call.message)

    elif call.data == "ask_ai":
        user_chat_mode[chat_id] = True
        bot.send_message(chat_id, "💬 Ask me anything!")


# ============================
# RUN BOT
# ============================
def run():
    while True:
        try:
            bot.polling(none_stop=True, interval=1)
        except Exception as e:
            print("Error:", e)
            time.sleep(5)


if __name__ == "__main__":
    run()
